In [2]:
from itertools import product

from math import inf
import numpy as np
import scipy.special
from sklearn.datasets import make_regression


def bfgs(f, f_grad, x0, max_iter=500, tol=1e-6):
    """
    BFGS algorithm for unconstrained optimization.

    Args:
        f: The objective function to be minimized.
        f_grad: The gradient of the objective function.
        x0: The initial guess for the solution.
        max_iter: The maximum number of iterations.
        tol: The tolerance for convergence.

    Returns:
        A tuple containing:
            - The optimal solution found.
            - The value of the objective function at the solution.
            - The number of iterations performed.
    """

    x = x0
    n = len(x0)
    B = np.eye(n)  # Initial approximation of the Hessian inverse
    grad = f_grad(x)

    for i in range(max_iter):
        # Compute search direction
        p = -np.linalg.solve(B, grad)

        # Perform line search
        t = line_search(f, f_grad, x, p)

        # Update x
        x_next = x + t * p

        # Update B (BFGS update)
        s = x_next - x
        y = f_grad(x_next) - grad
        if np.dot(s, y) > 0:  # Ensure positive definiteness
            rho = 1 / np.dot(s, y)
            B = (np.eye(n) - rho * np.outer(s, y)) @ B @ (np.eye(n) - rho * np.outer(y, s)) + rho * np.outer(s, s)
        else:
            B = np.eye(n)

        # Update x and gradient
        x = x_next
        grad = f_grad(x)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

    return x, f(x), i + 1


def line_search(f, f_grad, x, p, c1=1e-4, c2=0.9):
    """
    Backtracking line search.

    Args:
        f: The objective function.
        f_grad: The gradient of the objective function.
        x: The current point.
        p: The search direction.
        c1: The Armijo condition parameter.
        c2: The curvature condition parameter.

    Returns:
        The step size that satisfies the Wolfe conditions.
    """
    assert 0 < c1 < c2 < 1
    beta = inf
    alpha = 0.0
    t = 1.0
    s = np.dot(f_grad(x), p)

    for iter in range(10_000):
        # Check Armijo Condition
        if f(x + t * p) - f(x) <= c1 * t * s:
            # Check Wolfe Condition
            if f_grad(x + t * p) @ p >= c2 * s:
                break
            else:
                # Wolfe failed
                alpha = t
        else:
            # Armijo failed
            beta = t
        if beta < inf:
            t = (alpha + beta) / 2
        else:
            t = 2 * alpha

    if iter == 10_000 - 1:
        raise ValueError("Line search did not converge at x={x}, p={p}, t={t:.2e}".format(x=x, p=p, t=t))
    return t


# Example usage:
def f(x):
    return x[0] ** 2 + 5 * x[1] ** 4 + 0.1 * abs(x[0])


def f_grad(x, dx=1e-6):
    n = len(x)
    grad = np.zeros(n)
    f_val = f(x)
    for i in range(n):
        x_plus = x.copy()
        x_plus[i] += dx
        grad[i] = (f(x_plus) - f_val) / dx
    return grad


x0 = np.array([3.0, 4.0])
x_opt, f_opt, iterations = bfgs(f, f_grad, x0)

print("Optimal solution:", x_opt)
print("Optimal value:", f_opt)
print("Iterations:", iterations)

ValueError: Line search did not converge at x=[ 4.55014146e-08 -1.22990350e-01], p=[-1.46540516e+04  4.90101639e-02], t=6.21e-12

In [13]:
from scipy.optimize import minimize, minimize_scalar
from sklearn.datasets import make_regression

x, y = make_regression(n_samples=100, n_features=10, noise=10, n_informative=3, random_state=42)


def f(beta):
    # enet criterion
    y_hat = x @ beta
    mse = np.mean((y - y_hat) ** 2)
    penalty = 10.0 * np.sum(np.abs(beta)) + 5 * np.sum(beta ** 2)
    return mse + penalty


beta_0 = np.zeros(shape=(x.shape[1],))
result = minimize(f, beta_0, method='BFGS')
print(result)


  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 4095.9387337492435
        x: [ 1.869e+00 -3.943e-05  2.888e-06  1.031e+01 -8.730e-05
            -3.855e-06  3.466e-05  1.423e+00  6.406e-01 -5.678e-05]
      nit: 44
      jac: [-6.171e-02 -1.086e+01  1.628e+01 -2.887e-02 -8.228e+00
            -1.065e+01  2.563e+00  1.318e-02  6.421e-02 -3.080e-01]
 hess_inv: [[ 5.202e-02 -2.913e-05 ... -1.341e-03 -8.904e-03]
            [-2.913e-05  7.300e-07 ...  3.534e-06 -9.934e-05]
            ...
            [-1.341e-03  3.534e-06 ...  3.544e-02  6.536e-03]
            [-8.904e-03 -9.934e-05 ...  6.536e-03  1.052e-01]]
     nfev: 2088
     njev: 189


In [60]:
# GD with line search Enet
from scipy.optimize import approx_fprime, minimize_scalar


def f_enet(beta, l1: float, l2: float) -> float:
    # enet criterion
    y_hat = x @ beta
    mse = np.mean((y - y_hat) ** 2)
    penalty = l1 * np.sum(np.abs(beta)) + l2 * np.sum(np.square(beta))
    return mse + penalty


def f_grad_enet(beta, l1: float, l2: float) -> np.ndarray:
    grad_mse = x.T @ (x @ beta - y)
    grad_penalty = l1 * np.sign(beta) + l2 * beta
    return (grad_mse + grad_penalty) / x.shape[0]


def gd_with_line_search(l1: float, l2: float, max_iter=1_000, loss_rtol: float = 1e-8) -> np.ndarray:
    n, p = x.shape
    beta = np.zeros(p, dtype=np.float64)
    loss = f_enet(beta, l1, l2)
    mom = 0.9
    g = np.zeros_like(beta)

    for i in range(max_iter):
        grad = f_grad_enet(beta, l1, l2)
        g = (1 - mom) * grad + mom * g
        # grad = grad / np.linalg.norm(grad)
        p = -g

        t = 1.0

        new_beta = beta + t * p
        new_loss = f_enet(new_beta, l1, l2)

        if i % 100 == 0:
            print(f"Iteration {i:,}, loss {new_loss:.2e}, t {t:.2e}")

        beta = new_beta
        loss = new_loss

    return beta

# [0.562, -0.0, 0.0, 5.217, -0.0, -0.0, 0.0, 0.368, 0.0, -0.0]
b = gd_with_line_search(10.0, 10.0)
print([round(float(b_), 3) for b_ in b])


/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/1153051812.py:35: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x


Iteration 0, loss 4.42e+03, t 7.88e-01
Iteration 100, loss 4.42e+03, t 3.74e-09
Iteration 200, loss 4.42e+03, t 3.74e-09
Iteration 300, loss 4.42e+03, t 3.74e-09
Iteration 400, loss 4.42e+03, t 3.74e-09
Iteration 500, loss 4.42e+03, t 3.74e-09
Iteration 600, loss 4.42e+03, t 3.74e-09
Iteration 700, loss 4.42e+03, t 3.74e-09
Iteration 800, loss 4.42e+03, t 3.74e-09
Iteration 900, loss 4.42e+03, t 3.74e-09
[1.293, 0.031, -0.306, 5.357, -0.122, 0.033, 0.375, 1.149, 0.835, -0.443]


In [57]:
from sklearn.linear_model import ElasticNet


def saga_enet(l1: float, l2: float, tol: float = 1e-6, max_iter: int = 50_000) -> np.ndarray:
    n, p = x.shape
    grad = np.zeros_like(x)
    g = np.zeros(x.shape[1])
    beta = np.zeros(x.shape[1])
    lr = 1e-1
    lr_target = 1e-10
    lr_factor = (lr_target / lr) ** (1 / max_iter)
    n_print = max_iter // 10

    for i in range(max_iter):
        idx = np.random.randint(n)
        prev = grad[idx, :]
        g -= prev
        x_i = x[idx, :]
        y_i = y[idx]
        grad_i = (
                     # MSE grad
                         (x_i @ beta - y_i) * x_i
                         # L1 grad
                         + l1 * np.sign(beta)
                         # L2 grad
                         + l2 * beta
                 ) / n
        grad[idx, :] = grad_i
        g += grad_i
        beta -= g * lr

        if np.linalg.norm(g) < tol:
            print(f"Converged at iteration {i:,}")
            break

        if i % n_print == 0:
            print(f"Iteration {i:,} grad norm: {np.linalg.norm(g):.2e}, lr: {lr:.2e}")
        lr *= lr_factor

    return beta


for l1, l2 in product([0.1, 1.0, 10.0], [0.1, 1.0, 10.0]):
    beta_saga = saga_enet(l1, l2)
    beta_cd = ElasticNet(alpha=l1 + l2, l1_ratio=l1 / (l1 + l2), fit_intercept=False).fit(x, y).coef_
    beta_gd = gd_with_line_search(l1, l2)

    print(f"\nl1={l1}, l2={l2}, diff={np.linalg.norm(beta_saga - beta_cd) / np.linalg.norm(beta_cd):.3%}")
    for b, name in zip([beta_saga, beta_cd, beta_gd], ["SAGA", "CD", "GD + LS"]):
        print(f"{name: >10}: {[round(float(b_), 3) for b_ in b]}")

# beta = saga_enet()
# print([round(float(b), 3) for b in beta])


Iteration 0 grad norm: 1.20e+00, lr: 1.00e-01
Iteration 5,000 grad norm: 3.29e-05, lr: 1.26e-02
Converged at iteration 5,813
Iteration 0, loss 6.50e+02, t 7.89e-01
Converged at iteration 2, loss 5.15e+02, t 3.74e-09

l1=0.1, l2=0.1, diff=0.003%
      SAGA: [15.193, 0.399, 0.164, 57.463, 1.273, 0.205, -0.313, 8.553, 1.007, -1.499]
        CD: [15.193, 0.4, 0.163, 57.463, 1.273, 0.206, -0.314, 8.552, 1.007, -1.499]
   GD + LS: [16.328, 0.129, 0.222, 59.29, 1.432, 1.15, 0.753, 8.908, 2.316, -2.705]
Iteration 0 grad norm: 1.45e+00, lr: 1.00e-01
Iteration 5,000 grad norm: 1.85e-05, lr: 1.26e-02
Converged at iteration 5,797
Iteration 0, loss 2.37e+03, t 4.61e-01
Converged at iteration 5, loss 2.36e+03, t 3.36e-09

l1=0.1, l2=1.0, diff=0.001%
      SAGA: [8.356, 0.122, -0.56, 32.263, 0.15, 0.264, 0.839, 5.6, 2.438, -1.725]
        CD: [8.356, 0.121, -0.56, 32.263, 0.15, 0.264, 0.839, 5.6, 2.438, -1.725]
   GD + LS: [7.78, 0.183, -1.76, 32.058, -0.676, 0.211, 2.179, 6.798, 4.859, -2.609]
Itera

/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x
/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x
/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x
/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tol

Iteration 0, loss 7.39e+02, t 7.80e-01
Converged at iteration 2, loss 5.99e+02, t 3.74e-09

l1=1.0, l2=0.1, diff=0.000%
      SAGA: [14.224, 0.0, 0.0, 56.846, 0.394, -0.0, 0.0, 7.917, 0.237, -0.499]
        CD: [14.224, 0.0, -0.0, 56.846, 0.395, -0.0, -0.0, 7.917, 0.237, -0.499]
   GD + LS: [16.229, 0.127, 0.149, 59.106, 1.374, 1.122, 0.818, 8.958, 2.453, -2.738]
Iteration 0 grad norm: 2.71e+00, lr: 1.00e-01
Iteration 5,000 grad norm: 1.16e-01, lr: 1.26e-02
Iteration 10,000 grad norm: 6.06e-02, lr: 1.58e-03
Iteration 15,000 grad norm: 1.32e-01, lr: 2.00e-04
Iteration 20,000 grad norm: 1.64e-01, lr: 2.51e-05
Iteration 25,000 grad norm: 9.81e-02, lr: 3.16e-06
Iteration 30,000 grad norm: 1.15e-01, lr: 3.98e-07
Iteration 35,000 grad norm: 1.43e-01, lr: 5.01e-08
Iteration 40,000 grad norm: 1.20e-01, lr: 6.31e-09
Iteration 45,000 grad norm: 2.04e-01, lr: 7.94e-10


/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x
/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x


Iteration 0, loss 2.42e+03, t 4.56e-01
Converged at iteration 6, loss 2.42e+03, t 6.25e-09

l1=1.0, l2=1.0, diff=0.001%
      SAGA: [7.827, 0.0, -0.165, 31.934, 0.0, 0.0, 0.402, 5.194, 2.058, -1.235]
        CD: [7.827, 0.0, -0.165, 31.934, 0.0, 0.0, 0.403, 5.194, 2.058, -1.235]
   GD + LS: [7.713, 0.181, -1.74, 31.777, -0.667, 0.21, 2.156, 6.734, 4.809, -2.583]
Iteration 0 grad norm: 2.03e+00, lr: 1.00e-01
Iteration 5,000 grad norm: 5.25e-02, lr: 1.26e-02
Iteration 10,000 grad norm: 7.81e-02, lr: 1.58e-03
Iteration 15,000 grad norm: 1.25e-01, lr: 2.00e-04
Iteration 20,000 grad norm: 4.84e-02, lr: 2.51e-05
Iteration 25,000 grad norm: 6.16e-02, lr: 3.16e-06
Iteration 30,000 grad norm: 1.32e-01, lr: 3.98e-07
Iteration 35,000 grad norm: 1.42e-01, lr: 5.01e-08
Iteration 40,000 grad norm: 4.85e-02, lr: 6.31e-09
Iteration 45,000 grad norm: 9.58e-02, lr: 7.94e-10
Iteration 0, loss 4.32e+03, t 8.85e-02
Converged at iteration 3, loss 4.32e+03, t 3.74e-09

l1=1.0, l2=10.0, diff=0.000%
      SAGA

/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x


Iteration 25,000 grad norm: 1.79e+00, lr: 3.16e-06
Iteration 30,000 grad norm: 1.72e+00, lr: 3.98e-07
Iteration 35,000 grad norm: 1.86e+00, lr: 5.01e-08
Iteration 40,000 grad norm: 1.41e+00, lr: 6.31e-09
Iteration 45,000 grad norm: 2.10e+00, lr: 7.94e-10
Iteration 0, loss 2.91e+03, t 4.06e-01
Converged at iteration 6, loss 2.90e+03, t 8.02e-09

l1=10.0, l2=1.0, diff=0.001%
      SAGA: [2.683, 0.0, 0.0, 28.083, 0.0, -0.0, -0.0, 1.038, 0.0, 0.0]
        CD: [2.683, 0.0, -0.0, 28.084, 0.0, -0.0, 0.0, 1.038, 0.0, -0.0]
   GD + LS: [7.016, 0.16, -1.555, 28.862, -0.585, 0.192, 1.934, 6.09, 4.322, -2.329]
Iteration 0 grad norm: 6.04e-01, lr: 1.00e-01
Iteration 5,000 grad norm: 5.90e-01, lr: 1.26e-02
Iteration 10,000 grad norm: 1.16e+00, lr: 1.58e-03
Iteration 15,000 grad norm: 1.68e+00, lr: 2.00e-04
Iteration 20,000 grad norm: 1.72e+00, lr: 2.51e-05


/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x


Iteration 25,000 grad norm: 1.40e+00, lr: 3.16e-06
Iteration 30,000 grad norm: 1.33e+00, lr: 3.98e-07
Iteration 35,000 grad norm: 2.45e+00, lr: 5.01e-08
Iteration 40,000 grad norm: 1.91e+00, lr: 6.31e-09
Iteration 45,000 grad norm: 1.70e+00, lr: 7.94e-10
Iteration 0, loss 4.42e+03, t 7.88e-02
Converged at iteration 3, loss 4.42e+03, t 7.07e-09

l1=10.0, l2=10.0, diff=0.000%
      SAGA: [0.562, -0.0, 0.0, 5.217, -0.0, -0.0, 0.0, 0.368, 0.0, -0.0]
        CD: [0.562, 0.0, -0.0, 5.217, -0.0, 0.0, 0.0, 0.368, 0.0, -0.0]
   GD + LS: [1.293, 0.031, -0.306, 5.357, -0.122, 0.033, 0.375, 1.148, 0.835, -0.443]


/var/folders/25/l9j270ds30s7j_dj15xfs5t80000gn/T/ipykernel_32406/3626109192.py:32: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  t = minimize_scalar(backtracking_line_search, bounds=(0, 1), tol=loss_rtol).x


In [118]:
def adam_enet(beta_1: float = 0.98, beta_2: float = 0.99, max_iter: int = 200_000) -> np.ndarray:
    """
    Minimize the Elastic Net criterion using the Adam optimizer.

    Args:
      beta_1: Exponential decay rate for the first moment estimates.
      beta_2: Exponential decay rate for the second moment estimates.
      max_iter: Maximum number of iterations.

    Returns:
      An array of the estimated coefficients.
    """
    n, p = x.shape
    beta = np.zeros(p)
    m = np.zeros(p)
    v = np.ones(p)
    epsilon = 1e-8
    alpha = 0.001  # Learning rate

    for t in range(1, max_iter + 1):
        batch_idx = np.random.randint(n, size=32)
        batch = x[batch_idx, :]
        grad = f_grad_enet(batch, beta) / n
        m = beta_1 * m + (1 - beta_1) * grad
        v = beta_2 * v + (1 - beta_2) * np.square(grad)
        m_hat = m / (1 - beta_1 ** t)
        v_hat = v / (1 - beta_2 ** t)
        beta -= alpha * m_hat / (np.sqrt(v_hat) + epsilon)

        if t % (max_iter // 10) == 0:
            print(f"Iteration {t:,} grad norm: {np.linalg.norm(grad)}")

    return beta


beta = adam_enet()
print(np.round(beta, 3))

ValueError: operands could not be broadcast together with shapes (100,) (32,) 